<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://docs.unidata.ucar.edu/images/logos/unidata_logo_vertical_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

# NEXRAD Level 3 Radar
**Python-AWIPS Tutorial Notebook**

<div style="clear:both"></div>
</div>

---

<div style="float:right; width:250 px"><img src="../images/NEXRAD_Level3_Radar_preview.png" alt="NEXRAD Composite Reflectivity Example" style="height: 300px;"></div>


# Objectives

* Use python-awips to connect to an edex server
* Define and filter data request for radar data
* Plot NEXRAD 3 algorithm, precipitation, and derived products (not base data)

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#EDEX-Connection" data-toc-modified-id="EDEX-Connection-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDEX Connection</a></span></li><li><span><a href="#Investigate-Data" data-toc-modified-id="Investigate-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Investigate Data</a></span><ul class="toc-item"><li><span><a href="#Available-Locations" data-toc-modified-id="Available-Locations-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Available Locations</a></span></li><li><span><a href="#Available-Parameters" data-toc-modified-id="Available-Parameters-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Available Parameters</a></span></li><li><span><a href="#Radar-Product-IDs-and-Names" data-toc-modified-id="Radar-Product-IDs-and-Names-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Radar Product IDs and Names</a></span></li></ul></li><li><span><a href="#Function:-make_map()" data-toc-modified-id="Function:-make_map()-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Function: make_map()</a></span></li><li><span><a href="#Plot-the-Data!" data-toc-modified-id="Plot-the-Data!-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plot the Data!</a></span></li><li><span><a href="#See-Also" data-toc-modified-id="See-Also-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>See Also</a></span><ul class="toc-item"><li><span><a href="#Related-Notebooks" data-toc-modified-id="Related-Notebooks-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Related Notebooks</a></span></li><li><span><a href="#Additional-Documention" data-toc-modified-id="Additional-Documention-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Additional Documention</a></span></li></ul></li></ul></div>

## Imports

The imports below are used throughout the notebook.  Note the first import is coming directly from python-awips and allows us to connect to an EDEX server.  The subsequent imports are for data manipulation and visualization. 

In [ ]:
import warnings
from awips.dataaccess import DataAccessLayer
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

<a href="#top">Top</a>

---

## EDEX Connection

First we establish a connection to Unidata's public EDEX server. This sets the proper server on the **DataAccessLayer**, which we will use numerous times throughout the notebook.

In [ ]:
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest("radar")

<a href="#top">Top</a>

---

## Investigate Data

Now that we've created a new radar data request, let's take a look at what locations and parameters are available for our current request.

### Available Locations

We can take a look at what "locations" are available for our radar request.  For radar, we'll see that radar station names are returned when looking at the availalbe location names.

For this example we'll use Morehead City, NC as our region of interest.  You can easily look up other station IDs and where they are using [this NWS webpage](https://radar.weather.gov/station/KMHX/standard).

In [ ]:
available_locs = DataAccessLayer.getAvailableLocationNames(request)
available_locs.sort()
print(available_locs)

# Set our location to Morehead City (kmhx)
request.setLocationNames("kmhx")

### Available Parameters

Next, let's look at the parameters returned from the available parameters request.  If we look closely, we can see that some of the parameters appear different from the others.

In [ ]:
availableParms = DataAccessLayer.getAvailableParameters(request)
availableParms.sort()
print(availableParms)

### Radar Product IDs and Names

As we saw above, some parameters seem to be describing different things from the rest.  The DataAccessLayer has a built in function to parse the available parameters into the separate **Product IDs** and **Product Names**.  Here, we take a look at the two different arrays that are returned when parsing the *availableParms* array we just recieved in the previous code cell.

In [ ]:
productIDs = DataAccessLayer.getRadarProductIDs(availableParms)
productNames = DataAccessLayer.getRadarProductNames(availableParms)
print(productIDs)
print(productNames)

<a href="#top">Top</a>

---

## Function: make_map()

In order to plot more than one image, it's easiest to define common logic in a function.  Here, a new function called **make_map** is defined.  This function uses the [matplotlib.pyplot package (plt)](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.html) to create a figure and axis.  The coastlines (continental boundaries) are added, along with lat/lon grids.

In [ ]:
def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(16, 16),
                           subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.coastlines(resolution='50m')
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

<a href="#top">Top</a>

---

## Plot the Data!

Here we'll create a plot for each of the Radar Product Names from our *productNames* array from the [previous section](#Radar-Product-IDs-and-Names).

In [ ]:
# suppress a few warnings that come from plotting
warnings.filterwarnings("ignore",category =RuntimeWarning)
warnings.filterwarnings("ignore",category =UserWarning)

# Cycle through all of the products to try and plot each one
for prod in productNames:
    
    request.setParameters(prod)
    availableLevels = DataAccessLayer.getAvailableLevels(request)
    
    # Check the available levels, if there are none, then skip this product
    if availableLevels:
        request.setLevels(availableLevels[0])
    else:
        print("No levels found for " + prod)
        continue

    cycles = DataAccessLayer.getAvailableTimes(request, True)
    times = DataAccessLayer.getAvailableTimes(request)

    if times:
        print()
        response = DataAccessLayer.getGridData(request, [times[-1]])
        print("Recs : ", len(response))
        
        if response:
            grid = response[0]
        else:
            continue
        data = grid.getRawData()
        lons, lats = grid.getLatLonCoords()
        
        print('Time :', str(grid.getDataTime()))
        flat = np.ndarray.flatten(data)
        print('Name :', str(grid.getLocationName()))
        print('Prod :', str(grid.getParameter()))
        print('Range:' , np.nanmin(flat), " to ", np.nanmax(flat), " (Unit :", grid.getUnit(), ")")
        print('Size :', str(data.shape))
        print()

        cmap = plt.get_cmap('rainbow')
        bbox = [lons.min()-0.5, lons.max()+0.5, lats.min()-0.5, lats.max()+0.5]
        fig, ax = make_map(bbox=bbox)
        cs = ax.pcolormesh(lons, lats, data, cmap=cmap)
        cbar = fig.colorbar(cs, extend='both', shrink=0.5, orientation='horizontal')
        cbar.set_label(grid.getParameter() +" " + grid.getLevel() + " " \
                       + grid.getLocationName() + " (" + prod + "), (" + grid.getUnit() + ") " \
                       + "valid " + str(grid.getDataTime().getRefTime()))
        plt.show()

<a href="#top">Top</a>

---

## See Also


### Related Notebooks

* [Grid Levels and Parameters](https://unidata.github.io/python-awips/examples/generated/Grid_Levels_and_Parameters.html)


### Additional Documention

**python-awips**

- [DataAccessLayer.changeEDEXHost()](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.changeEDEXHost)
- [DataAccessLayer.newDataRequest()](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.newDataRequest)
- [DataAccessLayer.getRadarProductIDs()](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getRadarProductIDs)
- [DataAccessLayer.getRadarProductNames()](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getRadarProductNames)

**matplotlib**

- [matplotlib.pyplot()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html)
- [matplotlib.pyplot.axes()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axes.html)
- [matplotlib.pyplot.figure()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.figure.html)

<a href="#top">Top</a>

---